In [ ]:
!pip install folium pandas numpy geopy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import folium
import numpy as np
import requests
from geopy.geocoders import Nominatim
import json
import pandas as pd
from collections import Counter

In [ ]:
# fetching all the columns and rows from database
data = pd.read_csv('/content/data.csv')
lat = data['latitude']
lon = data['longitude']
prob = data['LandslideProb']
data

,short_term_rainfall,long_term_rainfall,elevation_relief,latitude,longitude,LandslideProb
0,3.380403,7.375425,1018,31.885923,76.589350,0.076923
1,5.881771,59.199465,251,19.353412,84.213217,0.798913
2,24.100000,22.300000,770,9.990300,76.789800,0.931034
3,17.300000,100.700000,476,24.271730,93.280393,0.798913
4,0.000000,1.900000,157,20.944738,85.156627,0.076923
...,...,...,...,...,...,...
1557,0.338856,1.422287,2671,31.717909,77.880861,0.000000
1558,0.994879,0.316553,202,22.193159,82.398940,0.000000
1559,0.293076,6.913200,1606,12.873663,76.729564,0.000000
1560,2.418525,24.726712,365,18.593358,76.856010,0.000000


In [ ]:
longitudes = data['longitude'].values
latitudes = data['latitude'].values
probabilities = data['LandslideProb'].values

In [ ]:
# lat & lon to districty name 1118
# API name https://nominatim.openstreetmap.org/reverse.php?lat=28.611066239545426&lon=88.33007812500001&zoom=6&format=jsonv2
districts_to_plot = []
for a in range (0 , 1561):
    response = requests.get("https://nominatim.openstreetmap.org/reverse.php?lat=" + str(latitudes[a]) + "&lon="+ str(longitudes[a]) + "&zoom=6&format=jsonv2")
    r = response.json()
    districts_to_plot.append(r['name'])


KeyError: ignored

In [ ]:
counts = Counter(districts_to_plot)
print(counts)

Counter({'Ramban district': 34, 'Kamrup Metropolitan': 32, 'Doda district': 30, 'Darjeeling District': 26, 'Kullu': 26, 'Uttarkashi district': 16, 'Thane': 16, 'Shimla district': 16, 'Chamoli district': 15, 'Lahul and Spiti': 15, 'Papum Pare': 15, 'Mandi': 14, 'Chamba district': 13, 'Aizwal': 13, 'Idukki': 12, 'Kohima': 12, 'Kangra': 12, 'Kishtwar district': 11, 'Senapati': 11, 'Reasi district': 10, 'Pune': 10, 'Dakshina Kannada district': 10, 'South Goa': 10, 'Kalimpong district': 10, 'Baramulla district': 9, 'Dima Hasao district': 9, 'Kinnaur district': 9, 'Kannur district': 9, 'The Nilgiris District': 9, 'Coimbatore District': 9, 'Mumbai Suburban': 9, 'Udhampur District': 8, 'Kozhikode district': 8, 'Leh District': 8, 'India': 8, 'Kathua District': 8, 'Ri-Bhoi': 7, 'Nagarkurnool': 7, 'Tehri Garhwal': 7, 'Belgaum district': 7, 'Salem District': 7, 'North Sikkim District': 7, 'Bangalore Urban': 7, 'Gangtok district': 7, 'Nandyal': 7, 'Ganderbal district': 6, 'Uttara Kannada district':

In [ ]:
# Download the GeoJSON data for Indian states
with open("/content/india_ds.geojson") as f:
    rr = json.load(f)

In [ ]:
# Define the threshold probability
some_number = 0.5

# Download the GeoJSON data for Indian states
url = '/content/india_ds.geojson'

# Create a map object using OpenStreetMaps
m = folium.Map(location=[19.0760, 72.8777], zoom_start=9, tiles='openstreetmap')

# Loop through the GeoJSON data and add each district as a polygon to the map if it is in the list
for i in range(0, 481):
    d_name = rr['features'][i]['properties']['DISTRICT']
    if d_name in districts_to_plot:
        if probabilities[i] >= some_number and probabilities[i] < 0.8:
            folium.GeoJson(rr['features'][i], 
                           style_function=lambda x: {'fillColor': '#ff726f', 'fillOpacity': 10 * probabilities[i], 'weight': 1},
                           tooltip=d_name + '\n' +str(probabilities[i])).add_to(m)
        elif probabilities[i] < some_number:
            folium.GeoJson(rr['features'][i], 
                           style_function=lambda x: {'fillColor': 'pink', 'fillOpacity': 10 * probabilities[i], 'weight': 1},
                           tooltip=d_name + '\n' +str(probabilities[i])).add_to(m)
        elif probabilities[i] >= 0.8:
            folium.GeoJson(rr['features'][i], 
                           style_function=lambda x: {'fillColor': 'red', 'fillOpacity': 10 * probabilities[i], 'weight': 1},
                           tooltip=d_name + '\n' +str(probabilities[i])).add_to(m)
for i in range(0,481):
  folium.GeoJson(rr['features'][i], 
                           style_function=lambda x: {'weight': 1, 'fillOpacity' : 0, 'color': 'black'}).add_to(m)
# Display the map
m